In [1]:
#!wget https://huggingface.co/mesolitica/fasttext-ms-id-en/resolve/main/model-16.bin
# !wget https://huggingface.co/mesolitica/fasttext-language-detection-bahasa-en/resolve/main/fasttext.bin
# !wget https://huggingface.co/mesolitica/fasttext-language-detection-bahasa-en/resolve/main/fasttext.ftz
#pip3 install fasttext

In [21]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           125Gi       9.8Gi        63Gi       2.0Mi        52Gi       114Gi
Swap:             0B          0B          0B


In [22]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from datasets import Audio
import re
from glob import glob
import torch
from silero_vad import load_silero_vad
import IPython.display as ipd
import mp
from streaming import LocalDataset
import soundfile as sf
from streaming import MDSWriter
import fasttext

In [4]:
model = fasttext.load_model('fasttext.ftz')

In [5]:
sr = 16000
audio = Audio(sampling_rate=sr)

In [6]:
files = sorted(glob('partition-audio/*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
len(files)

79

In [7]:
def extract_language_code(text):
    start = text.find("<|startoftranscript|>") + len("<|startoftranscript|>")
    
    start = text.find("<|", start) + 2
    end = text.find("|>", start)
    
    if start != -1 and end != -1:
        return text[start:end]
    return None

In [13]:
columns = {
    'audio_filename': 'str',
    'text':'str'
}
compression = None
hashes = 'sha1', 'xxh64'

In [14]:
def loop(files):
    result = []
    files, _ = files
    for f in files:
        data = []
        try:
            with open(f) as fopen:
                count = 0
                for line in fopen:
                    data.append(json.loads(line))
            index = int(f.split('-')[-1].replace('.jsonl', ''))

        except Exception as e:
            print(f"Error reading {f}: {e}")
            continue
                
        for entry in tqdm(data):
            
            audio_filename = entry['audio_filename'].replace('IMDA-STT/', '')
            
            entry['audio_filename'] = audio_filename
            
            if not os.path.exists(audio_filename):
                continue
                
            label = entry['text']
            
            main_text = re.sub(r'<\|.*?\|>', '', label)
            
            whisper_lang = extract_language_code(label)
                        
            output = model.predict(main_text.strip(), k = 2)
            
            if (whisper_lang == 'en') & (output[0][0] != '__label__english'):
                
                continue
            
            else:
                
                result.append(entry)
                
    return result
                

In [15]:
result = mp.multiprocessing(files, loop, cores = 32)

100%|██████████| 100000/100000 [00:04<00:00, 24111.25it/s]

 98%|█████████▊| 97708/100000 [00:05<00:00, 20983.74it/s]]

100%|██████████| 100000/100000 [00:05<00:00, 19518.86it/s]

100%|██████████| 100000/100000 [00:05<00:00, 19135.46it/s]

100%|██████████| 100000/100000 [00:02<00:00, 42393.11it/s]


In [16]:
len(result)

7664890

In [38]:
with MDSWriter(out=f'mosaic-whisper-filtered-fasttext', columns=columns, compression=compression, hashes=hashes) as out:
    for x in result:
        out.write(x) 